<a href="https://colab.research.google.com/github/LuccasTraumer/ibjjf-data/blob/main/Pan_Americano_IBJJF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
import json
import re

# URL base dos JSONs
base_url = "https://raw.githubusercontent.com/LuccasTraumer/ibjjf-data/main/pan-ibjjf/"
anos = list(range(1996, 2024))

# Mapeamento para normalizar faixas
mapa_faixas = {
    "white": "WHITE", "white belt": "WHITE",
    "blue": "BLUE", "blue belt": "BLUE",
    "purple": "PURPLE", "purple belt": "PURPLE",
    "brown": "BROWN", "brown belt": "BROWN",
    "black": "BLACK", "black belt": "BLACK",
    "coral": "CORAL", "coral belt": "CORAL",
    "red": "RED", "red belt": "RED"
}

# Mapeamento para normalizar sexo
mapa_sexo = {
    "male": "MALE", "m": "MALE", "masculino": "MALE",
    "female": "FEMALE", "f": "FEMALE", "feminino": "FEMALE"
}

dfs = []

for ano in anos:
    url = f"{base_url}{ano}.json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = json.loads(response.text)

        linhas = []
        for item in data:
            categoria_raw = item.get("categoria", "").strip()
            ranking = item.get("ranking", [])

            partes = [p.strip() for p in categoria_raw.split(" / ")]
            divisao = partes[0] if len(partes) > 0 else None
            faixa_raw = partes[1] if len(partes) > 1 else None
            sexo_raw = partes[2] if len(partes) > 2 else None
            categoria_peso = partes[3] if len(partes) > 3 else None

            # Normaliza faixa e sexo
            faixa = None
            sexo = None
            if faixa_raw:
                faixa_lower = faixa_raw.lower().replace("belt", "").strip()
                faixa = mapa_faixas.get(faixa_lower, faixa_raw.upper())
            if sexo_raw:
                sexo_lower = sexo_raw.lower().strip()
                sexo = mapa_sexo.get(sexo_lower, sexo_raw.upper())

            for posicao, atleta in enumerate(ranking, start=1):
                linhas.append({
                    "ano": ano,
                    "competicao": "Brasileiro IBJJF",
                    "divisao": divisao,
                    "faixa": faixa,
                    "sexo": sexo,
                    "categoria_peso": categoria_peso,
                    "posicao": posicao,
                    "nome": atleta.get("nome"),
                    "academia": atleta.get("academia")
                })

        df = pd.DataFrame(linhas)
        dfs.append(df)
        print(f"✅ {ano} carregado — {len(df)} atletas")

    except Exception as e:
        print(f"⚠️ Erro ao carregar {ano}: {e}")

dados_bjj = pd.concat(dfs, ignore_index=True)
print(f"\nTotal de registros: {len(dados_bjj)}")

# Preview dos dados
dados_bjj.head(10)


✅ 1996 carregado — 190 atletas
✅ 1997 carregado — 115 atletas
✅ 1998 carregado — 164 atletas
✅ 1999 carregado — 296 atletas
✅ 2000 carregado — 371 atletas
✅ 2001 carregado — 366 atletas
✅ 2002 carregado — 423 atletas
✅ 2003 carregado — 456 atletas
✅ 2004 carregado — 475 atletas
✅ 2005 carregado — 485 atletas
✅ 2006 carregado — 599 atletas
✅ 2007 carregado — 654 atletas
✅ 2008 carregado — 734 atletas
✅ 2009 carregado — 799 atletas
✅ 2010 carregado — 869 atletas
✅ 2011 carregado — 924 atletas
✅ 2012 carregado — 1090 atletas
✅ 2013 carregado — 1135 atletas
✅ 2014 carregado — 1152 atletas
✅ 2015 carregado — 1211 atletas
✅ 2016 carregado — 1284 atletas
✅ 2017 carregado — 1364 atletas
✅ 2018 carregado — 1573 atletas
✅ 2019 carregado — 1653 atletas
✅ 2020 carregado — 1403 atletas
✅ 2021 carregado — 1701 atletas
✅ 2022 carregado — 1889 atletas
✅ 2023 carregado — 1940 atletas

Total de registros: 25315


,ano,competicao,divisao,faixa,sexo,categoria_peso,posicao,nome,academia
0,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Rooster,1,Ricardo Brandão,Carlson RJ
1,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Rooster,2,Marcio Okamura,Equipe Paulista
2,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Rooster,3,Mauricio Silveira,Equipe Paulista
3,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Light Feather,1,Andre Mota,Lewis-Nova Uniao
4,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Light Feather,2,Leonardo Santos,Lewis-Nova Uniao
5,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Light Feather,3,Otavio,Tenis Clube
6,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Feather,1,Adriano Pereira,Luis Paulo
7,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Feather,2,Alex Oliveira,Jucao
8,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Feather,3,Gustavo Aragão,Gracie Barra
9,1996,Brasileiro IBJJF,Adult,BLUE,MALE,Light,1,Tiago de Freitas,Machado Redondo


In [2]:
academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "academia"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)
academias.head(20)


,sexo,academia,titulos
4,BLACK,Alliance,91
83,BLACK,Gracie Barra,91
79,BLACK,GF Team,54
86,BLACK,Gracie Humaita,49
9,BLACK,Atos Jiu-Jitsu,42
34,BLACK,CheckMat,38
135,BLACK,Nova União,25
28,BLACK,Carlson Gracie Team,23
20,BLACK,Brasa CTA,22
151,BLACK,R1NG BJJ,20


In [3]:
# Atletas mais recorrentes
atletas = (
    dados_bjj.groupby("nome")
    .size()
    .sort_values(ascending=False)
)
atletas.head(10)


,0
nome,
Roberto Godoi,29
Carlos Alberto Terrinha,24
Wellington Leal Dias,18
Georgina Staley,17
Rachel Noelle Morrison,16
Luiza Monteiro Moura da Costa,15
Kelly Gerald Johnson,15
Melissa Bentley,14
Jackson McVicker,14


In [4]:
# Evolução da quantidade de participantes por ano
import plotly.express as px

contagem_ano = (
    dados_bjj.groupby("ano")
    .size()
    .reset_index(name="quantidade")
)

px.bar(contagem_ano, x="ano", y="quantidade", title="Número de atletas registrados por ano")


In [5]:
# Total de participações por atleta (independente da colocação)
participacoes = (
    dados_bjj
    .groupby(["sexo", "nome"])
    .size()
    .reset_index(name="participacoes")
    .sort_values(["sexo", "participacoes"], ascending=[True, False])
)

# Top 10 masculinos e femininos
top_masculino = participacoes[participacoes["sexo"] == "MALE"].head(10)
top_feminino  = participacoes[participacoes["sexo"] == "FEMALE"].head(10)

print("🏋️‍♂️ Top 10 Atletas Masculinos que mais competiram:")
display(top_masculino)

print("🏋️‍♀️ Top 10 Atletas Femininos que mais competiram:")
display(top_feminino)


🏋️‍♂️ Top 10 Atletas Masculinos que mais competiram:


,sexo,nome,participacoes
10035,MALE,James Boran,14
9295,MALE,Eduardo Leitao,12
12267,MALE,Rodrigo Medeiros,12
8282,MALE,Asa Fuller,11
8655,MALE,Cassio Werneck,11
9302,MALE,Eduardo Rocha,11
9441,MALE,Fabio Leopoldo,11
12216,MALE,Roberto Godoi,11
8112,MALE,Alexandre Ribeiro,10
11621,MALE,Nicolai Holt,10


🏋️‍♀️ Top 10 Atletas Femininos que mais competiram:


,sexo,nome,participacoes
7697,FEMALE,Luciana Dias,9
7552,FEMALE,Gisele Braga,6
7496,FEMALE,Clarissa Andic,5
7565,FEMALE,Hillary Williams,5
7651,FEMALE,Kelly Paul,5
7666,FEMALE,Kyra Gracie,5
7698,FEMALE,Luciana Tavares,5
7725,FEMALE,Mariana Coelho,5
7545,FEMALE,Gabrielle Garcia,4
7377,FEMALE,Alexis Dufresne,3


In [6]:
# Apenas os campeões (posição = 1)
campeoes = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "nome"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)

# Top 10 de cada sexo
top_campeoes_masc = campeoes[campeoes["sexo"] == "MALE"].head(10)
top_campeoes_fem  = campeoes[campeoes["sexo"] == "FEMALE"].head(10)

print("🥇 Top 10 Campeões (Masculino):")
display(top_campeoes_masc)

print("🥇 Top 10 Campeãs (Feminino):")
display(top_campeoes_fem)


🥇 Top 10 Campeões (Masculino):


,sexo,nome,titulos
2729,MALE,Andre Galvao,7
2769,MALE,Asa Fuller,7
3017,MALE,Daniel Thomas,7
3113,MALE,Eduardo Leitao,7
3367,MALE,James Boran,7
2714,MALE,Alexandre Ribeiro,6
2720,MALE,Amal Easton,6
2819,MALE,Brad Rehor,6
2829,MALE,Braulio Estima,6
3235,MALE,Gary Casey,6


🥇 Top 10 Campeãs (Feminino):


,sexo,nome,titulos
2575,FEMALE,Luciana Dias,6
2565,FEMALE,Kyra Gracie,5
2529,FEMALE,Gisele Braga,4
2531,FEMALE,Hillary Williams,3
2557,FEMALE,Kelly Paul,3
2582,FEMALE,Mariana Coelho,3
2474,FEMALE,ANA LAURA CORDEIRO,2
2476,FEMALE,Alessandra Filha,2
2478,FEMALE,Alexis Dufresne,2
2489,FEMALE,Beatriz Mesquita,2


In [7]:
academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby(["sexo", "academia"])
    .size()
    .reset_index(name="titulos")
    .sort_values(["sexo", "titulos"], ascending=[True, False])
)
academias.head(10)


,sexo,academia,titulos
4,BLACK,Alliance,91
83,BLACK,Gracie Barra,91
79,BLACK,GF Team,54
86,BLACK,Gracie Humaita,49
9,BLACK,Atos Jiu-Jitsu,42
34,BLACK,CheckMat,38
135,BLACK,Nova União,25
28,BLACK,Carlson Gracie Team,23
20,BLACK,Brasa CTA,22
151,BLACK,R1NG BJJ,20


In [8]:
import plotly.express as px

px.bar(
    top_campeoes_masc,
    x="titulos",
    y="nome",
    orientation="h",
    title="🥇 Top 10 Campeões (Masculino)",
    text="titulos"
)

px.bar(
    top_campeoes_fem,
    x="titulos",
    y="nome",
    orientation="h",
    title="🥇 Top 10 Campeãs (Feminino)",
    text="titulos"
)


In [9]:
# Academias com mais campeões (1º lugar)
top_academias = (
    dados_bjj[dados_bjj["posicao"] == 1]
    .groupby("academia")
    .size()
    .sort_values(ascending=False)
    .head(10)
)
top_academias


,0
academia,
Gracie Barra,631
Alliance,546
Atos Jiu-Jitsu,243
Gracie Humaita,215
CheckMat,203
Carlson Gracie Team,187
GF Team,174
Ribeiro Jiu-Jitsu,153
Nova União,111
